## PIL module documentation

_class_ PIL.__PIL__(edge_retain=0.003,polar_threshold=0.50,radius=12,min_samples=5,mode='image')

Apply a PIL detection algorithm over 2D magnetic field image data.

__Parameters__:
    

__edge__\_ __retain__: ___float,optional, default: 0.003___